In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
os.chdir('/kaggle/input/tensorflow-great-barrier-reef')
os.listdir()
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train_df = pd.read_csv("train.csv")
df_train_df

In [ ]:
##Checking for duplicates
df_train_df.duplicated().sum()

In [ ]:
###checking frames with annotations
df_train_df[df_train_df.annotations.str.len() > 2]

In [ ]:
## Visualizing the annotations
import ast
ast.literal_eval(df_train_df.iloc[16].annotations)

## Videos

In [ ]:
## Verify in there is corrupted data

In [ ]:
from os import listdir
from PIL import Image
def validate_images(video_id):
    path = 'train_images/video_{}/'.format(video_id) 
    print("Verifying that video {} frames are valid...".format(video_id))
    for filename in listdir(path):
        if filename.endswith('.jpg'):
            try:
                img = Image.open(path+filename)
                img.verify() 
            except (IOError, SyntaxError) as e:
                print('Bad file:', filename) # Print out the names of corrupt files
    print("Verified! Video {} has all valid images".format(video_id))

for video_id in range(3):
    validate_images(video_id)
    

In [ ]:
### Loading Sequences of images with annotations
from PIL import Image, ImageDraw

def fetch_image_list(df_tmp, video_id, num_images, start_frame_idx):
    def fetch_image(frame_id):
        path_base = 'train_images/video_{}/{}.jpg'
        raw_img = Image.open(path_base.format(video_id, frame_id))

        row_frame = df_tmp[(df_tmp.video_id == video_id) & (df_tmp.video_frame == frame_id)].iloc[0]
        bounding_boxes = ast.literal_eval(row_frame.annotations)

        for box in bounding_boxes:
            draw = ImageDraw.Draw(raw_img)
            x0, y0, x1, y1 = (box['x'], box['y'], box['x']+box['width'], box['y']+box['height'])
            draw.rectangle( (x0, y0, x1, y1), outline=180, width=3)
        return raw_img

    return [np.array(fetch_image(start_frame_idx + index)) for index in range(num_images)]

images = fetch_image_list(df_train_df, video_id = 0, num_images = 80, start_frame_idx = 25)

print("Num images: ", len(images))
plt.imshow(images[0], interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
##visualizing list of images as animations
from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_animation(ims):
    fig = plt.figure(figsize=(9, 9))
    plt.axis('off')
    im = plt.imshow(ims[0])

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//12)

create_animation(images)